[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/fund-census-ncen-python.ipynb)

# Analyze SEC Fund Census (N-CEN) Filings with Python -- Free, No API Key

Use **edgartools** to analyze annual fund census data from SEC N-CEN filings -- completely free, no API key or paid subscription required. Every registered investment company files an N-CEN annually, disclosing fund series, service providers, board composition, broker commissions, ETF mechanics, and securities lending.

**What you'll learn:**
- Parse an N-CEN filing to see all fund series in a fund family
- Explore the service provider network: advisers, custodians, transfer agents
- Analyze broker-dealer relationships and commissions
- Examine board of directors composition
- Access ETF-specific data: authorized participants, creation units, exchanges

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Parse a Fund Census Filing

Vanguard files an annual N-CEN covering all of its fund series. Parse the latest filing to see the full census:

In [ ]:
# Find the latest N-CEN filing for Vanguard
filings = get_filings(form="N-CEN").filter(company_name="VANGUARD")
census = filings[0].obj()

print(f"Registrant:      {census.name}")
print(f"CIK:             {census.cik}")
print(f"Report Date:     {census.report_date}")
print(f"Classification:  {census.classification_type}")
print(f"Fund Series:     {census.num_series}")
print(f"Is ETF Company:  {census.is_etf_company}")
print(f"Directors:       {len(census.registrant.directors)}")

## Rich Display

The `FundCensus` object has a built-in Rich display showing registrant info, all fund series, and the board of directors:

In [ ]:
census

## Fund Series

Each registered investment company can have multiple fund series. Get them all as a DataFrame with average net assets, fund type, and ETF status:

In [ ]:
series = census.series_data()
print(f"Total series: {len(series)}")

# Show the largest series by average net assets
top_series = series.sort_values("avg_net_assets", ascending=False).head(10).copy()
top_series["avg_net_assets"] = top_series["avg_net_assets"].apply(
    lambda v: f"${float(v)/1e9:.1f}B" if pd.notna(v) and float(v) > 1e9
    else f"${float(v)/1e6:.0f}M" if pd.notna(v) else ""
)
top_series[["name", "series_id", "fund_type", "avg_net_assets", "has_etf"]]

## Service Providers

N-CEN filings disclose the full service provider network for each fund series -- advisers, custodians, transfer agents, administrators, and pricing services:

In [ ]:
providers = census.service_providers()
print(f"Total provider relationships: {len(providers)}")

# Count providers by role
role_counts = providers.groupby("role").agg(
    relationships=("provider_name", "count"),
    unique_providers=("provider_name", "nunique")
).sort_values("relationships", ascending=False)
role_counts

## Broker Commissions

See which broker-dealers execute trades for the fund and how much commission each one earns:

In [ ]:
brokers = census.broker_data()
if not brokers.empty:
    print(f"Broker relationships: {len(brokers)}")

    # Top brokers by commission
    top_brokers = brokers.groupby("broker_name")["commission"].sum().sort_values(ascending=False).head(10)
    top_brokers = top_brokers.apply(lambda v: f"${float(v):,.0f}" if pd.notna(v) else "")
    top_brokers.to_frame("Total Commission")
else:
    print("No broker data in this filing")

## Board of Directors

See the fund's board composition, including which directors are "interested persons" (affiliated with the investment adviser):

In [ ]:
directors = census.director_data()
print(f"Total directors: {len(directors)}")
print(f"Independent:     {len(directors[~directors['interested_person']])}")
print(f"Interested:      {len(directors[directors['interested_person']])}")
print()

directors[["name", "interested_person"]]

## ETF-Specific Data

For fund families with ETFs, the N-CEN includes exchange, ticker, creation unit size, and authorized participant details:

In [ ]:
etfs = census.etf_data()
if not etfs.empty:
    print(f"ETF series: {len(etfs)}")
    etfs[["series_name", "ticker", "exchange", "creation_unit_size", "num_authorized_participants"]].head(10)
else:
    print("No ETF series in this filing")

## Compare Fund Families

Pull N-CEN data from several fund families to compare their size and structure:

In [ ]:
fund_families = ["VANGUARD", "FIDELITY", "ISHARES"]
rows = []

for family in fund_families:
    filing = get_filings(form="N-CEN").filter(company_name=family)
    if len(filing) > 0:
        c = filing[0].obj()
        rows.append({
            "Registrant": c.name[:45],
            "Series": c.num_series,
            "Directors": len(c.registrant.directors),
            "Has ETFs": c.is_etf_company,
            "Classification": c.classification_type or "N/A",
        })

pd.DataFrame(rows)

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing N-CEN fund census data:

**With edgartools (free, no API key):**
```python
census = filing.obj()
census.series_data()          # All fund series as DataFrame
census.service_providers()    # Full provider network
census.broker_data()          # Broker commissions
census.director_data()        # Board composition
census.etf_data()             # ETF mechanics
```

**Typical approach (manual XML parsing):**
```python
from lxml import etree
# ... download N-CEN XML from EDGAR,
# ... strip namespaces, handle 280+ XML elements,
# ... parse nested series → providers → brokers hierarchy,
# ... clean 'N/A' sentinels, handle missing elements
```

With edgartools, N-CEN data is parsed into 12 structured models -- fund series, providers, brokers, directors, and ETF info all accessible as pandas DataFrames.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse an N-CEN filing ──
filing = get_filings(form="N-CEN").filter(company_name="VANGUARD")[0]
census = filing.obj()

# ── Registrant metadata ──
census.name                    # Registrant name
census.cik                     # SEC CIK
census.report_date             # Report period end date
census.classification_type     # N-1A, N-2, etc.
census.num_series              # Number of fund series
census.is_etf_company          # Has any ETF series?

# ── DataFrames ──
census.series_data()           # Fund series with net assets, type
census.service_providers()     # Advisers, custodians, transfer agents
census.broker_data()           # Broker-dealer commissions
census.director_data()         # Board members, independence
census.etf_data()              # Exchange, ticker, creation units

# ── Registrant details ──
census.registrant.directors    # List of Director objects
census.registrant.accountant   # Accountant (name, PCAOB #)
census.registrant.phone        # Contact phone
```

## What's Next

You've learned how to analyze fund census data from SEC N-CEN filings. Here are related tutorials:

- [Money Market Fund Holdings (N-MFP)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/money-market-fund-nmfp-python.ipynb)
- [Mutual Fund Holdings (N-PORT)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/mutual-fund-holdings-nport-python.ipynb)
- [ETF and Fund Holdings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/etf-fund-holdings-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who analyzes fund operations? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*